<b>PARA QUE AS VARIÁVEIS SEJAM DEVIDAMENTE INICIALIZADAS E OS RESULTADOS CORRETAMENTE EXIBIDOS, RODE TODAS AS CELLS EM:<br> CELL -> RUN ALL<b>

In [44]:
from ipycanvas import Canvas, hold_canvas
import pandas as pd
import numpy as np

In [45]:
path = ""
my_name = "CG2020_T1_GabrielBoscoli"

In [46]:
cie_table = pd.read_excel(path + 'all_1nm_data.xls', header=3)

In [47]:
# Le as colunas em listas
lambda_ = cie_table['nm']
Lw = cie_table['CIE D65']
x_ = cie_table['x bar']
y_ = cie_table['y bar']
z_ = cie_table['z bar']

lambda_ = np.asarray(lambda_[80:481])
Lw = np.asarray(Lw[80:481])
x_ = np.asarray(x_[80:481])
y_ = np.asarray(y_[80:481])
z_ = np.asarray(z_[80:481])

Lw = Lw / np.amax(Lw)

In [48]:
checker = pd.read_excel(path + 'ColorChecker_RGB_and_spectra.xls',
                        'spectral_data',
                        skiprows=1,
                        nrows=24)
reflectance = checker.loc[:, checker.columns[2:]].to_numpy(dtype=np.float32)
colors = checker['Color name']

In [49]:
Lw_ = Lw[(lambda_ > 379) & (lambda_ < 731) & (lambda_ % 10 == 0)]
xbar = x_[(lambda_ > 379) & (lambda_ < 731) & (lambda_ % 10 == 0)]
ybar = y_[(lambda_ > 379) & (lambda_ < 731) & (lambda_ % 10 == 0)]
zbar = z_[(lambda_ > 379) & (lambda_ < 731) & (lambda_ % 10 == 0)]

In [50]:
def print_colors_code(colorsCode):
    for i in range(len(colorsCode)):
        print(f'{colors[i]} -> ({colorsCode[i][0]}, {colorsCode[i][1]}, {colorsCode[i][2]})')

In [51]:
k = 1.0 / (10 * np.sum(Lw_ * ybar))

XYZ = np.zeros(shape=(colors.size, 3))
for i, cor in enumerate(colors):
    betha = reflectance[i, :]
    XYZ[i, 0] = 10 * k * np.sum(Lw_ * betha * xbar)
    XYZ[i, 1] = 10 * k * np.sum(Lw_ * betha * ybar)
    XYZ[i, 2] = 10 * k * np.sum(Lw_ * betha * zbar)
    
print_colors_code(XYZ)

dark skin -> (0.11113817162661657, 0.10032432852432753, 0.06713859773589231)
light skin -> (0.3730351931222515, 0.34720840720495194, 0.25399301119894574)
blue sky -> (0.1768360771622641, 0.18877469174506856, 0.3470212026518497)
foliage -> (0.10534128950735919, 0.13295357932346522, 0.06977566291411123)
blue flower -> (0.2486386280855693, 0.23494163275289445, 0.44183395071423764)
bluish green -> (0.30991693155277367, 0.4267889967641575, 0.4499488492879537)
orange -> (0.3696832295997508, 0.2956037417870296, 0.062353869788027094)
purplish blue -> (0.1348799873882133, 0.11766618725675312, 0.38533097159670654)
moderate red -> (0.27759634037669717, 0.18798375999626069, 0.13624753068864773)
purple -> (0.0834891193313974, 0.06322822676043566, 0.1402977429413308)
yellow green -> (0.3364506796971514, 0.44260124471664886, 0.11314971633597846)
orange yellow -> (0.4526036985488901, 0.4209649924440995, 0.07821637195843865)
blue -> (0.07961298282093976, 0.06021575145123009, 0.2832056903908718)
green -

In [52]:
def gamma(val):
    if (val < 0):
        return 0
    elif (val < 0.0031308):
        return 12.92 * val
    else:
        return 1.055 * val**(1.0 / 2.4) - 0.055


def CIEXYZ_to_sRGB(XYZ):
    Minv = np.array([[3.2404542, -1.5371385, -0.4985314],
                     [-0.9692660, 1.8760108, 0.0415560],
                     [0.0556434, -0.2040259, 1.0572252]])
    rgb = np.dot(Minv, XYZ)
    sRGB = [gamma(i) for i in rgb]
    return sRGB

In [53]:
sRGB = np.zeros(shape=(colors.size, 3))
for i in range(colors.size):
    sRGB[i] = CIEXYZ_to_sRGB([XYZ[i][0], XYZ[i][1], XYZ[i][2]])
    sRGB[i] = [round(i * 255) for i in sRGB[i]]

sRGB = sRGB.astype(int)
np.save("resp/" + my_name + ".npy", sRGB)
print_colors_code(sRGB)

dark skin -> (115, 81, 67)
light skin -> (195, 149, 129)
blue sky -> (93, 123, 157)
foliage -> (90, 108, 65)
blue flower -> (130, 129, 176)
bluish green -> (99, 191, 171)
orange -> (220, 123, 45)
purplish blue -> (72, 92, 168)
moderate red -> (195, 84, 98)
purple -> (90, 59, 105)
yellow green -> (160, 189, 62)
orange yellow -> (229, 161, 41)
blue -> (43, 62, 147)
green -> (71, 149, 72)
red -> (176, 48, 56)
yellow -> (238, 200, 22)
magenta -> (188, 84, 150)
cyan -> (0, 136, 166)
white 9.5 (.05 D) -> (245, 245, 240)
neutral 8 (.23 D) -> (200, 202, 201)
neutral 6.5 (.44 D) -> (160, 161, 161)
neutral 5 (.70 D) -> (120, 121, 121)
neutral 3.5 (1.05 D) -> (83, 84, 85)
black 2 (1.5 D) -> (50, 50, 50)


In [54]:
def round_rect(canvas, x, y, w, h, r):
    canvas.fill_style = "black"
    canvas.begin_path()
    r = min(r, w / 2, h / 2)
    canvas.move_to(x + r, y)
    canvas.arc_to(x + w, y, x + w, y + h, r)
    canvas.arc_to(x + w, y + h, x, y + h, r)
    canvas.arc_to(x, y + h, x, y, r)
    canvas.arc_to(x, y, x + w, y, r)
    canvas.close_path()
    canvas.fill()

In [55]:
w = 710
h = 510
canvas = Canvas(width=w, height=h, sync_image_data=True)

In [56]:
def save_to_file(*args, **kwargs):
    canvas.save()
    canvas.to_file("resp/" + my_name + ".png")

In [57]:
border_pad = 30
square_side = 100
pad_between_quares = 10
square_columns = 6
square_lines = 4
name_pad = 50
name = "Gabriel Boscoli"

canvas.observe(save_to_file, 'image_data')

round_rect(canvas, 0, 0, w, h, 20)
canvas.fill_style = 'white'

x = border_pad
y = border_pad
sRGB_index = 0
for i in range(square_lines):
    for j in range(square_columns):
        color = f'rgb({sRGB[sRGB_index][0]},{sRGB[sRGB_index][1]},{sRGB[sRGB_index][2]})'
        canvas.fill_style = color
        canvas.fill_rect(x, y, square_side, square_side)
        x += square_side + pad_between_quares
        sRGB_index += 1
    y += square_side + pad_between_quares
    x = border_pad

canvas.font = '30px serif'
canvas.text_align = "center"
canvas.text_baseline = "middle"
canvas.fill_style = "white"
canvas.fill_text(name, w / 2, h - (name_pad / 2))
canvas

Canvas(height=510, sync_image_data=True, width=710)